In [8]:
from models import *
import torch
import torchtext
from torchtext import data
import numpy as np

import spacy

#Hyperparameters 
learning_rate = 0.001
batch_size = 64
epochs = 25
seed = 0

torch.manual_seed(seed)

def tokenizer(text):
    spacy_en = spacy.load('en')
    return [tok.text for tok in spacy_en(text)]

def result(output):
    if baseline_output > 0.5:
        result = "subjective"
    else: 
        result = "objective"
    return result
    

In [9]:
# ------------------------------------------------------------------------------------
# Load models 
baseline = torch.load('models/model_baseline.pt')
baseline.eval()
cnn = torch.load('models/model_cnn.pt')
cnn.eval()
rnn = torch.load('models/model_rnn.pt')
rnn.eval()

# Instantiates 2 data.Field objects 
TEXT = data.Field(sequential=True,lower=True, tokenize='spacy', include_lengths=True)
LABELS = data.Field(sequential=False, use_vocab=False)

# Load the train, validation, and test datasets to become datasets
train_data, val_data, test_data = data.TabularDataset.splits(
        path='data/', train='train.tsv',
        validation='validation.tsv', test='test.tsv', format='tsv',
        skip_header=True, fields=[('text', TEXT), ('label', LABELS)])

# Create an object that can be enumerated (for training loop later)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
  (train_data, val_data, test_data), batch_sizes=(batch_size, batch_size, batch_size),
sort_key=lambda x: len(x.text), device=None, sort_within_batch=True, repeat=False)


# Vocab object contains the index/token for each unique word in the dataset (looks through all sentences in dataset)
TEXT.build_vocab(train_data, val_data, test_data)

# Loading GloVe Vector and Using Embedding Layer
TEXT.vocab.load_vectors(torchtext.vocab.GloVe(name='6B', dim=100))
vocab = TEXT.vocab

# ------------------------------------------------------------------------------------
print("Welcome to the subjective bot!\nPlease enter a sentence with more than 4 words (all in lower case)")
print("To exit the loop, enter 'exit' as the input")

while True:
    # Get input sentence 
    sentence = input("\nEnter a sentence\n")
    if sentence == "exit":
        break
    else:
        # Convert sentence string into token (using SpaCy)
        tokens = tokenizer(sentence) #list of individual string words

        # Convert each string token to an integer
        token_ints = [vocab.stoi[tok] for tok in tokens] #list of individual numbers

        # Convert the list of token integers into a torch.LongTensor with the shape [L,1] <--- L is number of tokens/words
        token_tensor = torch.LongTensor(token_ints).view(-1,1) # Shape is [sentence_len, 1]

        # Create a tensor for the length of the sentence with the shape [1]
        lengths = torch.Tensor([len(token_ints)]) #needed when calling RNN 

        #TEXT.build_vocab(token_tensor)
        #TEXT.vocab.load_vectors(torchtext.vocab.GloVe(name='6B', dim=100))

        #------------------------------------- MODELS ------------------------------------------
        baseline_output = np.round(torch.sigmoid(baseline(token_tensor)).detach().numpy(),3)
        baseline_result = result(baseline_output)
        print("Model baseline:", baseline_result, baseline_output)

        rnn_output = np.round(rnn(token_tensor, lengths).detach().numpy(),3)
        rnn_result = result(rnn_output)
        print("Model rnn:", rnn_result, rnn_output)

        cnn_output = np.round(cnn(token_tensor).detach().numpy(),3)
        cnn_result = result(cnn_output)
        print("Model cnn:", cnn_result, cnn_output)


Welcome to the subjective bot!
Please enter a sentence with more than 4 words (all in lower case)
To exit the loop, enter 'exit' as the input

Enter a sentence
the dog has nice fur 
Model baseline: subjective [0.581]
Model rnn: subjective [0.01]
Model cnn: subjective [0.792]

Enter a sentence
the clouds are white today
Model baseline: subjective [0.679]
Model rnn: subjective [0.004]
Model cnn: subjective [0.007]

Enter a sentence
that movie was really terrible
Model baseline: subjective [0.989]
Model rnn: subjective [0.999]
Model cnn: subjective [1.]

Enter a sentence
these flowers smell very nice
Model baseline: subjective [0.994]
Model rnn: subjective [0.997]
Model cnn: subjective [1.]

Enter a sentence
exit
